## Notebook to compare total demand by TAZ for the truck mode in two different scenarios

In [ ]:
# Notebook to compare total demand by TAZ for the truck mode in two different scenarios

import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

In [ ]:
# Root directory for MoDX output for "base year" model results.
#
base_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Root directory for MoDX output for "comparison scenario" model results.
# 
comparison_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'

### User input required: Path to user's "sandbox" directory

In [ ]:
# ===>>> USER INPUT REQUIRED: <<<===
# 
# 1. Supply path to root of user's "sandbox" directory:
#
my_sandbox_dir = r'S:/my_modx_output_dir/'

### User input required: CSV report file for "base" scenario

In [ ]:
# 2. Supply name of CSV file with report output for "base" scenario:
#
base_csv_fn = 'taz_truck_report_base.csv'

In [ ]:
### User input required: CSV report file for "comparison" scenario

In [ ]:
# 3. Supply name of CSV file with report for "comparison" scenario:
#
comp_csv_fn = 'taz_truck_report_comp.csv'

### User input required: CSV report file for this notebook

In [ ]:
# 4. Supply name of CSV output file for report generated by this notebook:
#
csv_output_fn = 'taz_transit_report_DIFF.csv'

In [ ]:
fq_base_csv_fn = my_sandbox_dir + base_csv_fn
fq_comp_csv_fn = my_sandbox_dir + comp_csv_fn

In [ ]:
# Columns containing "reference" data
#
ref_column_names = [ 'id', 'town', 'state' ]
#
# Columns containing "metric" data
#
metric_column_names = [ 'truck_total', 'heavy_truck', 'medium_truck', 'light_truck' ]
#
all_column_names = ref_column_names + metric_column_names

### Read "base" scenario data

In [ ]:
# Read "base" data into a dataframe
#
base_df = pd.read_csv(fq_base_csv_fn, delimiter=',', usecols=all_column_names)

In [ ]:
base_df

### Read "comparison" scenario data

In [ ]:
# Read "comparison" data into a dataframe
comp_df = pd.read_csv(fq_comp_csv_fn, delimiter=',',  usecols=all_column_names)

In [ ]:
comp_df

In [ ]:
# Create a dataframe in which to compute the delta between "comp" and "base"
delta_df = pd.merge(left=comp_df, right=base_df, on="id", suffixes=('_comp', '_base'))

In [ ]:
delta_df

### Compute delta: comparison - base

In [ ]:
# Compute the detlta between the two dataframes: comp - base
# Calculate the delta (comparision - base) for each metric
#
for column_name in metric_column_names:
    base_column_name = column_name + '_base'
    comp_column_name = column_name + '_comp'
    delta_column_name = column_name + '_delta'
    delta_df[delta_column_name] = delta_df[comp_column_name] - delta_df[base_column_name]
# end_for

In [ ]:
delta_df

In [ ]:
taz_shapefile_base_dir = r'G:/Data_Resources/modx/canonical_TAZ_shapefile/'

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(delta_df.set_index('id'), on='id')

### Export report output

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = my_sandbox_dir + csv_output_fn
delta_column_names = [ cn + '_delta' for cn in metric_column_names]
cols = ref_column_names + delta_column_names
joined_df.to_csv(fq_output_fn, sep=',', columns=cols)

### Make static and interactive maps of results

In [ ]:
# Make a static map of total auto trips by origin TAZ
joined_df.plot("truck_total_delta", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Change in Total Truck Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='truck_total_delta', 
                 geo=True, 
                 hover_cols=cols, 
                 clabel='Change in Total Truck Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Change in Total Truck Trips by Origin TAZ')